In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time

%matplotlib inline

import difi
from difi import __version__
print("difi version: {}".format(__version__))

difi version: 1.1.dev40+gc5afefd


In [2]:
DATA_DIR = "../data"

In [3]:
observations = pd.read_csv(
    os.path.join(DATA_DIR, "observations.txt"), 
    index_col=False, 
    sep=" "
)

In [4]:
column_mapping = {
    "obs_id" : "obs_id",
    "truth" : "obj_id",
    "time" : "mjd_utc",
    "night" : "night",
    "linkage_id" : "trkSub"
}

In [5]:
def simulateSSP(observations, 
                known_objects=None,
                linking_window_size=15, 
                attribution=True, 
                precovery=True,
                column_mappling=column_mapping):
    """
    Simulate the discovery perforance of Rubin Obs / LSST's 
    Solar System Pipelines (SSP).
    
    
    """
    # Track the objects linked
    #obj_ids_linked = np.array([])

    # Track "known" objects
    if known_objects is None:
        known_obj_ids = np.array([])
    else:
        known_obj_ids = known_objects
    
    # Track the observation IDs linked
    obs_ids_linked = np.array([])
    
    # If attribution is assumed, track attributed observations
    attributed_obs_ids = np.array([])

    # If precovery is assumed, track attributed observations
    precovery_obs_ids = np.array([])
    
    linkage_id_start = 0
    linkage_members_list = []
    all_linkages_list = []

    for i, night_end in enumerate(observations[column_mapping["night"]].unique()):

        print("Night ({}/{}): {}".format(
            i + 1,
            observations[column_mapping["night"]].nunique(),
            night_end
            )
        )
        
        if attribution:
            attribution_mask = (
                (observations[column_mapping["night"]] == night_end)
                & (observations[column_mapping["truth"]].isin(known_obj_ids))
            )
            attributed_obs_ids_iter = observations[attribution_mask][column_mapping["obs_id"]].values

        else: 
            attributed_obs_ids_iter = np.array([])
        
        # Only consider observations inside the linking window
        linking_window_mask = (
            (observations[column_mapping["night"]] >= (night_end - linking_window_size))
            & (observations[column_mapping["night"]] <= night_end)
        )
        
        # Remove already linked observations and observations attributed to
        # previous discoveries
        observations_mask = (
            linking_window_mask
            & (~observations[column_mapping["obs_id"]].isin(obs_ids_linked))
            & (~observations[column_mapping["obs_id"]].isin(precovery_obs_ids))
            & (~observations[column_mapping["obs_id"]].isin(attributed_obs_ids))
        )

        print("Observations between night {} and {}: {}".format(
            observations[observations_mask][column_mapping["night"]].min(),
            night_end,
            len(observations[observations_mask])
            ) 
        )

        # Find what should be findable up to the end night
        all_truths, findable_observations, summary = difi.analyzeObservations(
            observations[observations_mask],
            metric="nightly_linkages",
            linkage_min_obs=2,
            min_linkage_nights=3,
            max_obs_separation=1.5/24,
            column_mapping=column_mapping
        )

        if len(findable_observations) > 0:

            # Convert findable observations into a linkage_members data frame
            linkage_members = findable_observations["obs_ids"].apply(pd.Series).stack().to_frame(name="obs_id")
            linkage_members.reset_index(
                level=1, 
                drop=True,
                inplace=True
            )

            # Rename new index to trkSub
            linkage_members.index.name = "trkSub"
            linkage_members.reset_index(
                inplace=True,
                drop=False
            )
            # Convert trkSub from an integer to a unique string identifier
            linkage_members["trkSub"] = linkage_members["trkSub"].apply(
                lambda x: "t{:07d}".format(x + linkage_id_start)
            )
            linkage_members_list.append(linkage_members)

            # Create all_linkages data frame
            trkSubs = linkage_members["trkSub"].unique()
            all_linkages = pd.DataFrame({
                "trkSub" : trkSubs,
                "submission_night" : [night_end for i in range(len(trkSubs))]
            })
            all_linkages_list.append(all_linkages)

            # Increase linkage ID start value
            linkage_id_start += len(trkSubs)

            # Add linked observations to running list
            obs_ids_linked_iter = linkage_members[column_mapping["obs_id"]].values
            obj_ids_linked_iter = findable_observations[column_mapping["truth"]].values

            if precovery:
                precovery_mask = (
                    linking_window_mask
                    & (~observations[column_mapping["obs_id"]].isin(obs_ids_linked_iter))
                    & (observations[column_mapping["truth"]].isin(obj_ids_linked_iter))
                )
                precovery_obs_ids_iter = observations[precovery_mask][column_mapping["obs_id"]].values
            else:
                precovery_obs_ids_iter = np.array([])

        else:
            obs_ids_linked_iter = np.array([])
            obj_ids_linked_iter = np.array([])
            precovery_obs_ids_iter = np.array([])
            attributed_obs_ids_iter = np.array([])

        obs_ids_linked = np.concatenate([obs_ids_linked, obs_ids_linked_iter])
        precovery_obs_ids =  np.concatenate([precovery_obs_ids, precovery_obs_ids_iter])
        attributed_obs_ids =  np.concatenate([attributed_obs_ids, attributed_obs_ids_iter])
        known_obj_ids = np.concatenate([known_obj_ids, obj_ids_linked_iter])
        print("Observations linked: {}".format(len(obs_ids_linked_iter)))
        print("Observations precovered: {}".format(len(precovery_obs_ids_iter)))
        print("Observations attributed: {}".format(len(attributed_obs_ids_iter)))
        print("Cumulative observations linked: {}".format(len(obs_ids_linked)))
        print("Cumulative observations precovered: {}".format(len(precovery_obs_ids)))
        print("Cumulative observations attributed: {}".format(len(attributed_obs_ids)))
        print("")

    all_linkages = pd.concat(all_linkages_list)
    linkage_members = pd.concat(linkage_members_list)
    
    return all_linkages, linkage_members

In [6]:
all_linkages, linkage_members = simulateSSP(observations)

Night (1/20): 59854
Observations between night 59854 and 59854: 122355
Observations linked: 0
Observations precovered: 0
Observations attributed: 0
Cumulative observations linked: 0
Cumulative observations precovered: 0
Cumulative observations attributed: 0

Night (2/20): 59855
Observations between night 59854 and 59855: 242974
Observations linked: 0
Observations precovered: 0
Observations attributed: 0
Cumulative observations linked: 0
Cumulative observations precovered: 0
Cumulative observations attributed: 0

Night (3/20): 59856
Observations between night 59854 and 59856: 286624
Observations linked: 0
Observations precovered: 0
Observations attributed: 0
Cumulative observations linked: 0
Cumulative observations precovered: 0
Cumulative observations attributed: 0

Night (4/20): 59857
Observations between night 59854 and 59857: 504154
Observations linked: 653
Observations precovered: 0
Observations attributed: 0
Cumulative observations linked: 653
Cumulative observations precovered: 0

In [7]:
all_linkages.to_csv(
    os.path.join(DATA_DIR, "all_linkages.txt"), 
    index=False, 
    sep=" "
)

linkage_members.to_csv(
    os.path.join(DATA_DIR, "linkage_members.txt"), 
    index=False, 
    sep=" "
)